In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [91]:
import pickle
import pandas as pd
import json
import boto3
import sys

/Users/vist/.local/share/virtualenvs/analysis-j_VNnu_c/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [3]:
data = pickle.load(open("apthunt.pkl", 'rb'))

In [4]:
# To go from dynamo format to python
from boto3.dynamodb import types

deserializer = types.TypeDeserializer()
python_data = [{k: deserializer.deserialize(v) for k,v in d.items() if k == "parsed_thumbs"} for d in data]

In [5]:
del data

In [6]:
df = pd.DataFrame(python_data)

In [7]:
del python_data

In [8]:
thumbs = df["parsed_thumbs"]

In [9]:
del df

In [38]:
sqs = boto3.resource('sqs')
queue = sqs.get_queue_by_name(QueueName='apthunt-thumbs')

In [24]:
t = thumbs[1]

In [25]:
sqs.send_message(
        QueueUrl=queue_url,
        MessageBody=json.dumps(t)
    )

{'MD5OfMessageBody': 'fbfaf7ac0cec7e08644410acdecd3101',
 'MessageId': '0220241d-123f-4847-a195-163e7bccb94e',
 'ResponseMetadata': {'RequestId': '6bd10008-b8ae-5258-9394-56b0931e5956',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6bd10008-b8ae-5258-9394-56b0931e5956',
   'date': 'Sat, 08 Feb 2020 03:22:24 GMT',
   'content-type': 'text/xml',
   'content-length': '378'},
  'RetryAttempts': 0}}

In [41]:
def paginate(l, size=10):
    for i in range(0, len(l), size):
        yield l[i:i+size]

In [110]:
l = len(thumbs)
page = 10 # found by experiment.

prepared = [{"Id": f"{i}", "MessageBody": json.dumps(t)} for i, t in enumerate(thumbs) if t is not None]
for n, t in enumerate(paginate(prepared, page)):
    processed = n * page
    if processed%10000 == 0: print(f"{processed}/{l} ({processed/l*100:.2f}%)")

    queue.send_messages(Entries=t)
    
#     sqs.send_message(
#         QueueUrl=queue_url,
#         MessageBody=json.dumps(t)
#     )

0/619289 (0.00%)
10000/619289 (1.61%)
20000/619289 (3.23%)
30000/619289 (4.84%)
40000/619289 (6.46%)
50000/619289 (8.07%)
60000/619289 (9.69%)
70000/619289 (11.30%)
80000/619289 (12.92%)
90000/619289 (14.53%)
100000/619289 (16.15%)
110000/619289 (17.76%)
120000/619289 (19.38%)
130000/619289 (20.99%)
140000/619289 (22.61%)
150000/619289 (24.22%)
160000/619289 (25.84%)
170000/619289 (27.45%)
180000/619289 (29.07%)
190000/619289 (30.68%)
200000/619289 (32.30%)
210000/619289 (33.91%)
220000/619289 (35.52%)
230000/619289 (37.14%)
240000/619289 (38.75%)
250000/619289 (40.37%)
260000/619289 (41.98%)
270000/619289 (43.60%)
280000/619289 (45.21%)
290000/619289 (46.83%)
300000/619289 (48.44%)
310000/619289 (50.06%)
320000/619289 (51.67%)
330000/619289 (53.29%)
340000/619289 (54.90%)
350000/619289 (56.52%)
360000/619289 (58.13%)
370000/619289 (59.75%)
380000/619289 (61.36%)
390000/619289 (62.98%)
400000/619289 (64.59%)
410000/619289 (66.20%)
420000/619289 (67.82%)
430000/619289 (69.43%)
440000/61